### Summary
Record mean 1 row of the csv file. 

This notebook converts the records into alarms. 

In [1]:
import pandas as pd
from pprint import pprint
import gc # for forcing garbage collection
import time
from alarms import  convertRecordsToAlarmsV1
from alarms import  _convertRecordsToAlarmsOld
from alarms import maskSourceNames
start = time.time()

In [2]:
files = ["haziran2019.csv","march2019.csv","mayis2019.csv","nisan2019.csv"]
# files = ["mayis2019.csv"]
files = ["formatted-pre-1-"+f for f in files]
cols = ["MachineName","SourceName","EventTime", "Message","MessageType","Quality","Condition","Mask","NewState","Status"]
path = "./data/new/"

In [3]:
# for f in files:    
#     print("==================== File : {} =============".format(f))
#     input_fname = f
#     output_fname = "formatted-pre-2-" + input_fname.split("-")[-1]
#     df = pd.read_csv(path + input_fname, low_memory=False, usecols=cols ,parse_dates=["EventTime"])
#     print(df.info())
#     alarms = []
#     print("Total Number of Sources :  ", len(df["SourceName"].unique()))
#     print("Staring Processing records of each source : ", end= " ")
#     index = 0 # only for printing 
#     for sname in df["SourceName"].unique():
#         records =  df.loc[df['SourceName'].isin([sname])]
#         records = records.to_dict(orient="records")
#         temp_alarms = convertRecordsToAlarms(records)
#         print("({},{},{})".format(index,sname, len(temp_alarms)), end=" ")
#         alarms += temp_alarms # concat list
#         index += 1
        
#     print("\n\n Done: All records are converted to to alarms.")
#     # writing to files
#     df2 = pd.DataFrame(alarms)
#     df2.to_csv(path + output_fname, index = False)
#     print("Processing Time = {} minutes".format((time.time()-start)/60.0))
#     print(df2)

# print(">Done: Total Time to Complete : ", time.time() - start)

In [4]:
for f in files:    
    print("==================== File : {} =============".format(f))
    input_fname = f
    output_fname = "formatted-pre-2-" + input_fname.split("-")[-1]
    df = pd.read_csv(path + input_fname, low_memory=False, usecols=cols ,parse_dates=["EventTime"])
    print(df.info())
    alarms = []
    differs = []
    sources_ranks_dict = df['SourceName'].value_counts()
    id = 0
    for sname in sources_ranks_dict.keys():
        id += 1
        df_sname = df.loc[df['SourceName'].isin([sname])] # source DF
        types_rank_dict = df_sname["MessageType"].value_counts() # source ranks
        total = 0
        for key in types_rank_dict.keys():
            # print(key, types_rank_dict[key], end=",")
            total += types_rank_dict[key]
        assert(total== sources_ranks_dict[sname]) # sum is equal to count 
        # if len(types_rank_dict.keys()) > 2:
        #     print(">> Source: {}".format(sname, sources_ranks_dict[sname]), end="=>")
        #     print(types_rank_dict, end="")

        stime = time.time()
        temp_alarms1 = convertRecordsToAlarmsV1(df_sname)
        # print("Time1", time.time()-stime, end=",")
        alarms += temp_alarms1
        stime = time.time()
        temp_alarms2 = _convertRecordsToAlarmsOld(df_sname.to_dict(orient="records"))
        # print("Time2", time.time()-stime, end=",")
        if len(temp_alarms1)-len(temp_alarms2) != 0:
            print(">>[{}]Source: {},Conditions:{}".format(id,sname, df_sname["Condition"].unique()), end="=>")
            print("ALARMS1:{},Alarms2:{},Diff(new-old):{}".format(len(temp_alarms1),len(temp_alarms2), len(temp_alarms1)-len(temp_alarms2)),end="")
            print("")

        
        
        if len(temp_alarms2) > 0 and len(temp_alarms2) == 0:
            assert(False)
        # assert(len(temp_alarms1) == len(temp_alarms2)) # will not equal  
        if (len(temp_alarms1) != len(temp_alarms2)):
            differs.append(sname)

        # writing to files
    df2 = pd.DataFrame(alarms)
    # maskSourceNames(df2)
    df2.to_csv(path + output_fname, index = False)
    # print("Processing Time = {} minutes".format((time.time()-start)/60.0))
    print("Differs in 2 Algos",differs,len(differs))
    print(">>>>>",df2.info())
print(">Done: Total Time to Complete : ", time.time() - start)

==================== File : formatted-pre-1-haziran2019.csv =============
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1839157 entries, 0 to 1839156
Data columns (total 10 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   EventTime    datetime64[ns]
 3   Message      object        
 4   Quality      int64         
 5   Condition    object        
 6   Mask         int64         
 7   NewState     int64         
 8   Status       int64         
 9   MessageType  object        
dtypes: datetime64[ns](1), int64(4), object(5)
memory usage: 140.3+ MB
None
>>[1]Source: 48TIC2026,Conditions:['VEL-' 'VEL+']=>ALARMS1:578148,Alarms2:578192,Diff(new-old):-44
>>[2]Source: 47TI931A,Conditions:['IOP']=>ALARMS1:224209,Alarms2:224226,Diff(new-old):-17
>>[3]Source: 19LI105A,Conditions:['VEL-' 'LO' 'LL' 'VEL+']=>ALARMS1:37016,Alarms2:37013,Diff(new-old):3
>>[4]Source: 47TI870A,Conditions:['HTRP' 'HHH']=

# Section Debugging

In [5]:
# print(differs,len(differs))
# sname = "47TAL002CD-ANN"
# df_source = df.loc[df['SourceName'].isin([sname])]

# alarms1 = convertRecordsToAlarmsV1(df_source)
# alarms2 = convertRecordsToAlarmsV2(df_source.to_dict(orient="records"))
# len(alarms1), len(alarms2)

In [6]:


# #######

# s= "47TAL003AB-ANN"
# df3 = pd.read_csv(path + files[0], low_memory=False, usecols=cols ,parse_dates=["EventTime"]) # may file
# temp_df=df3.loc[df3["SourceName"].isin([s])]

# alarms1 = convertRecordsToAlarmsV1(temp_df)
# alarms2 = _convertRecordsToAlarmsOld(temp_df.to_dict(orient="records"))


# temp_df2 = temp_df.sort_values(by=["EventTime"])

# temp_df2.to_csv(path+"debug.csv")
# len(alarms1), len(alarms2)

In [7]:
# # for index, row in temp_df2.iterrows():
# #     print(row['EventTime'], row['Condition'], row["MessageType"])
# temp_df2

In [8]:
# print(s)
# df4 = pd.read_csv(path + "formatted-pre-2-mayis2019.csv", low_memory=False)
# df4.loc[df4["SourceName"].isin([s])]